In [51]:
import pandas as pd
import plotly.express as px
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import plotly.graph_objs as go




In [52]:
# Specify the column names
column_names = ['id', 'platform', 'sentiment', 'text']

df = pd.read_csv('twitter_training.csv', names=column_names)




In [53]:
df.head()

,id,platform,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         74682 non-null  int64 
 1   platform   74682 non-null  object
 2   sentiment  74682 non-null  object
 3   text       73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [55]:
df.describe()

,id
count,74682.000000
mean,6432.586165
std,3740.427870
min,1.000000
25%,3195.000000
50%,6422.000000
75%,9601.000000
max,13200.000000


In [56]:
df.dropna(inplace=True)  # Drop rows with missing values

df.isnull().sum()

id           0
platform     0
sentiment    0
text         0
dtype: int64

In [57]:
sentiment_counts = df['sentiment'].value_counts()
sentiment_counts

sentiment
Negative      22358
Positive      20655
Neutral       18108
Irrelevant    12875
Name: count, dtype: int64

In [67]:
# Create a pie chart for sentiment distribution
fig = px.pie(sentiment_counts, 
             values=sentiment_counts.values, 
             names=sentiment_counts.index, 
             title='Distribution of Sentiments in Social Media Data')

fig.update_layout(
    template='plotly_dark',
    width=600,
        title={
        'text': 'Distribution of Sentiments in Social Media Data',
        'x': 0.5,  # Set x to 0.5 for center alignment
        'xanchor': 'center',  # Anchor the title to the center
        'yanchor': 'top'  # Align the title to the top of the plot area
    }
)

# Show the plot
fig.show()

In [68]:
# Group by platform and sentiment to get counts
platform_sentiment_counts = df.groupby(['platform', 'sentiment']).size().reset_index(name='count')

# Create a bar chart for sentiment distribution by platform
fig = px.bar(platform_sentiment_counts, x='platform', y='count', color='sentiment',
             labels={'count': 'Count', 'platform': 'Platform', 'sentiment': 'Sentiment'},
             title='Sentiment Distribution by Platform')

# Show the plot

fig.update_layout(
    template='plotly_dark',
        title={
        'text': 'Sentiment Distribution by Platform',
        'x': 0.5,  # Set x to 0.5 for center alignment
        'xanchor': 'center',  # Anchor the title to the center
        'yanchor': 'top'  # Align the title to the top of the plot area
    }
)

fig.show()


In [60]:
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiment polarity using VADER
def get_sentiment_vader(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

# Apply sentiment analysis using VADER to the 'text' column
df['sentiment_score'] = df['text'].apply(get_sentiment_vader)

# Classify sentiment based on VADER compound score
def classify_sentiment(score):
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['sentiment'] = df['sentiment_score'].apply(classify_sentiment)

# Display the sentiment analysis results


In [61]:
platform_sentiment = df.groupby('platform')['sentiment_score'].mean().reset_index()
platform_sentiment['sentiment'] = platform_sentiment['sentiment_score'].apply(classify_sentiment)

# Display average sentiment scores per platform
print(platform_sentiment)


                             platform  sentiment_score sentiment
0                              Amazon         0.268199  Positive
1                         ApexLegends         0.112404  Positive
2                      AssassinsCreed         0.247627  Positive
3                         Battlefield        -0.099417  Negative
4                         Borderlands         0.212614  Positive
5                               CS-GO         0.179534  Positive
6                          CallOfDuty         0.018756   Neutral
7           CallOfDutyBlackopsColdWar        -0.005481   Neutral
8                       Cyberpunk2077         0.169034  Positive
9                               Dota2         0.095302  Positive
10                               FIFA         0.003969   Neutral
11                           Facebook        -0.063885  Negative
12                           Fortnite         0.041960   Neutral
13                             Google         0.076911  Positive
14                GrandTh

In [62]:

# Create a list of unique platforms for dropdown options
platforms = platform_sentiment['platform'].unique()

# Create an empty figure to hold all gauge charts
fig = go.Figure()

# Create each gauge chart for sentiment scores by platform
for index, row in platform_sentiment.iterrows():
    platform = row['platform']
    sentiment_score = row['sentiment_score']
    sentiment = row['sentiment']
    
    # Create the gauge chart
    gauge_chart = go.Indicator(
        mode="gauge+number",
        value=sentiment_score,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': f"Sentiment Score for {platform}"},
        gauge={
            'axis': {'range': [-1, 1]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [-1, -0.05], 'color': "red"},
                {'range': [-0.05, 0.05], 'color': "gray"},
                {'range': [0.05, 1], 'color': "green"}
            ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': 0
            }
        },
        visible=False  # Initially hide all charts
    )
    
    # Add the gauge chart to the figure
    fig.add_trace(gauge_chart)

# Make the first gauge chart visible initially
fig.data[0].visible = True

# Create dropdown menu for platform selection
buttons = []
for i, platform in enumerate(platforms):
    visibility = [i == j for j in range(len(platforms))]
    button = dict(
        label=platform,
        method="update",
        args=[{"visible": visibility}, {"title": f"Sentiment Scores for {platform}"}],
    )
    buttons.append(button)

# Update layout for the figure with dropdown menu
fig.update_layout(
    title='Sentiment Scores by Platform',
    template='plotly_dark',
    updatemenus=[
        dict(
            type="dropdown",
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.1,
            y=1.1,
        )
    ]
)

# Show the plot
fig.show()
